# Uploading Batch Input File
 - batch file 보낼 때 너무 많이 보내지 말고 구분해서 보내기

In [1]:
from getpass import getpass

openai_api_key = getpass("OPENAI_API_KEY")

In [2]:
from openai import OpenAI
import json
import pandas as pd

client = OpenAI(api_key=openai_api_key)

In [3]:
batch_input_file = client.files.create(
    file=open("PreviewQuiz_Structure_High_batch.jsonl","rb"),
    purpose="batch"
)

print(batch_input_file)

FileObject(id='file-8h79Jch9o28MGjqfc62NCk', bytes=2378854, created_at=1739777617, filename='PreviewQuiz_Structure_High_batch.jsonl', object='file', purpose='batch', status='processed', status_details=None)


# Create the Batch

In [4]:
# 업로드한 배치 입력 파일 ID 저장
batch_input_file_id = batch_input_file.id
print(batch_input_file_id)

file-8h79Jch9o28MGjqfc62NCk


In [5]:
batch_info = client.batches.create(
    input_file_id = batch_input_file_id,
    endpoint = "/v1/chat/completions", # 챗봇 api
    completion_window = "24h", # 현재는 이 기능만 가능
    metadata = {
        "description": "preview quiz"
    }
)

In [6]:
response_json = json.loads(batch_info.json())
response_output = json.dumps(response_json, ensure_ascii=False, indent = 2)
print(response_output)

{
  "id": "batch_67b2e6567c948190934c21239f7a0c47",
  "completion_window": "24h",
  "created_at": 1739777622,
  "endpoint": "/v1/chat/completions",
  "input_file_id": "file-8h79Jch9o28MGjqfc62NCk",
  "object": "batch",
  "status": "validating",
  "cancelled_at": null,
  "cancelling_at": null,
  "completed_at": null,
  "error_file_id": null,
  "errors": null,
  "expired_at": null,
  "expires_at": 1739864022,
  "failed_at": null,
  "finalizing_at": null,
  "in_progress_at": null,
  "metadata": {
    "description": "preview quiz"
  },
  "output_file_id": null,
  "request_counts": {
    "completed": 0,
    "failed": 0,
    "total": 0
  }
}


# Checking the Status of Batch
 - validating : the input file is being validated before the batch can begin
 - failed : the input file has failed the validation process
 - in_progress : the input file was successfully validated and the batch is currently being run
 - finalizing : the batch has completed and the results are being prepared
 - completed : the batch has been completed and the results are ready
 - expired : the batch was not able to be completed within the 24-hour time window
 - cancelling : the batch is being cancelled (may take up to 10 minutes)
 - cancelled : the batch was cancelled

- batch_id = 'batch_67b2e6567c948190934c21239f7a0c47' : previewquiz_structure_High

In [9]:
batch_id = "batch_67b2e6567c948190934c21239f7a0c47"
batch = client.batches.retrieve(batch_id)
batch_json = json.loads(batch.json())
batch_json

{'id': 'batch_67b2e6567c948190934c21239f7a0c47',
 'completion_window': '24h',
 'created_at': 1739777622,
 'endpoint': '/v1/chat/completions',
 'input_file_id': 'file-8h79Jch9o28MGjqfc62NCk',
 'object': 'batch',
 'status': 'completed',
 'cancelled_at': None,
 'cancelling_at': None,
 'completed_at': 1739778047,
 'error_file_id': None,
 'errors': None,
 'expired_at': None,
 'expires_at': 1739864022,
 'failed_at': None,
 'finalizing_at': 1739778023,
 'in_progress_at': 1739777623,
 'metadata': {'description': 'preview quiz'},
 'output_file_id': 'file-45JVYJ8CzvHvn79PPyK6hv',
 'request_counts': {'completed': 329, 'failed': 0, 'total': 329}}

In [10]:
batch_json['completed_at']

1739778047

## 시간 확인

In [43]:
from datetime import datetime
import pytz

kst = pytz.timezone('Asia/Seoul')

# 변환 함수
def convert_timestamp(starttime, endtime):
    st = datetime.utcfromtimestamp(starttime)
    end = datetime.utcfromtimestamp(endtime)
    dt_kst_st = st.replace(tzinfo=pytz.utc).astimezone(kst)
    dt_kst_end = end.replace(tzinfo=pytz.utc).astimezone(kst)

    return f"시작 시간 : {dt_kst_st}, 끝나는 시간 : {dt_kst_end}"

convert_timestamp(starttime = batch_json['created_at'], endtime = batch_json['completed_at'])


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_18092\1843218754.py:8: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  st = datetime.utcfromtimestamp(starttime)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_18092\1843218754.py:9: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  end = datetime.utcfromtimestamp(endtime)


'시작 시간 : 2025-02-13 23:07:06+09:00, 끝나는 시간 : 2025-02-13 23:21:58+09:00'

# Retrieving the Results
 - file-EVCbFfNykgV9HphK2h9wsk

In [11]:
file_response = client.files.content('file-45JVYJ8CzvHvn79PPyK6hv') # output_file_id 이용
print(file_response.text)

{"id": "batch_req_67b2e7e7a9788190bc4de7e4031d6822", "custom_id": "request-1", "response": {"status_code": 200, "request_id": "f25bea8514ca367e95764e3506e49e9d", "body": {"id": "chatcmpl-B1pvOhQYyKGyJdmSzwhyUZzi4Tsts", "object": "chat.completion", "created": 1739777626, "model": "o3-mini-2025-01-31", "choices": [{"index": 0, "message": {"role": "assistant", "content": "{\n  \"goal\": [\n    \"\uc804\uce58\uc0ac\uc758 \ubaa9\uc801\uc5b4\ub85c\uc11c\uc758 \ub3d9\uba85\uc0ac\ub97c \uc0ac\uc6a9\ud560 \uc218 \uc788\uc5b4\uc694.\"\n  ],\n  \"preview_quizzes\": [\n    {\n      \"fill_in_blank_quiz\": \"I enjoy _____ in team sports.\",\n      \"korean_translation\": \"\ub098\ub294 \ud300 \uc2a4\ud3ec\uce20\uc5d0\uc11c \ub178\ub294 \uac83\uc744 \uc990\uaca8\uc694.\",\n      \"options\": [\n        \"play\",\n        \"playing\",\n        \"played\"\n      ],\n      \"answer\": \"playing\",\n      \"answer_index\": 2\n    },\n    {\n      \"fill_in_blank_quiz\": \"She is excited about _____ goal

In [12]:
sample_list = file_response.text.split("\n")
sample_list = [item for item in sample_list if item.strip()]
sample_list

['{"id": "batch_req_67b2e7e7a9788190bc4de7e4031d6822", "custom_id": "request-1", "response": {"status_code": 200, "request_id": "f25bea8514ca367e95764e3506e49e9d", "body": {"id": "chatcmpl-B1pvOhQYyKGyJdmSzwhyUZzi4Tsts", "object": "chat.completion", "created": 1739777626, "model": "o3-mini-2025-01-31", "choices": [{"index": 0, "message": {"role": "assistant", "content": "{\\n  \\"goal\\": [\\n    \\"\\uc804\\uce58\\uc0ac\\uc758 \\ubaa9\\uc801\\uc5b4\\ub85c\\uc11c\\uc758 \\ub3d9\\uba85\\uc0ac\\ub97c \\uc0ac\\uc6a9\\ud560 \\uc218 \\uc788\\uc5b4\\uc694.\\"\\n  ],\\n  \\"preview_quizzes\\": [\\n    {\\n      \\"fill_in_blank_quiz\\": \\"I enjoy _____ in team sports.\\",\\n      \\"korean_translation\\": \\"\\ub098\\ub294 \\ud300 \\uc2a4\\ud3ec\\uce20\\uc5d0\\uc11c \\ub178\\ub294 \\uac83\\uc744 \\uc990\\uaca8\\uc694.\\",\\n      \\"options\\": [\\n        \\"play\\",\\n        \\"playing\\",\\n        \\"played\\"\\n      ],\\n      \\"answer\\": \\"playing\\",\\n      \\"answer_index\\": 2

In [13]:
sample_output = []

for sample in sample_list:
    for i in range(len(sample_list)):
        sample_json = json.loads(sample)
        content = sample_json["response"]["body"]["choices"][0]["message"]["content"]
        content_json = json.loads(content)
        sample_output.append(f"{i} 번째")
        sample_output.append(content_json)
        print(json.dumps(content_json, indent=2, ensure_ascii=False))
        break

# print(json.dumps(sample_output))


{
  "goal": [
    "전치사의 목적어로서의 동명사를 사용할 수 있어요."
  ],
  "preview_quizzes": [
    {
      "fill_in_blank_quiz": "I enjoy _____ in team sports.",
      "korean_translation": "나는 팀 스포츠에서 노는 것을 즐겨요.",
      "options": [
        "play",
        "playing",
        "played"
      ],
      "answer": "playing",
      "answer_index": 2
    },
    {
      "fill_in_blank_quiz": "She is excited about _____ goals in matches.",
      "korean_translation": "그녀는 경기에서 골 넣는 것에 신나해요.",
      "options": [
        "scoring",
        "score",
        "scored"
      ],
      "answer": "scoring",
      "answer_index": 1
    }
  ],
  "sample_script": [
    {
      "sample_script_eng": "A: What are you excited about these days? B: I am excited about dancing at festivals. A: Why is he nervous about the contest? B: He is nervous about competing in tournaments.",
      "sample_script_kor": "A: 요즘 무엇에 신나해요? B: 축제에서 춤추는 것에 신나해요. A: 왜 그 사람은 대회에 대해 긴장해요? B: 그는 토너먼트에 참가하는 것에 대해 긴장해요."
    }
  ]
}
{
  "goal": [
    "명사를 수

# batch 결과

In [14]:
for sample in sample_list:
    sample_json = json.loads(sample)
    content = sample_json["response"]["body"]["choices"][0]["message"]["content"]
    content_json = json.loads(content)
    print(json.dumps(content_json, indent=2, ensure_ascii=False))
    break

{
  "goal": [
    "전치사의 목적어로서의 동명사를 사용할 수 있어요."
  ],
  "preview_quizzes": [
    {
      "fill_in_blank_quiz": "I enjoy _____ in team sports.",
      "korean_translation": "나는 팀 스포츠에서 노는 것을 즐겨요.",
      "options": [
        "play",
        "playing",
        "played"
      ],
      "answer": "playing",
      "answer_index": 2
    },
    {
      "fill_in_blank_quiz": "She is excited about _____ goals in matches.",
      "korean_translation": "그녀는 경기에서 골 넣는 것에 신나해요.",
      "options": [
        "scoring",
        "score",
        "scored"
      ],
      "answer": "scoring",
      "answer_index": 1
    }
  ],
  "sample_script": [
    {
      "sample_script_eng": "A: What are you excited about these days? B: I am excited about dancing at festivals. A: Why is he nervous about the contest? B: He is nervous about competing in tournaments.",
      "sample_script_kor": "A: 요즘 무엇에 신나해요? B: 축제에서 춤추는 것에 신나해요. A: 왜 그 사람은 대회에 대해 긴장해요? B: 그는 토너먼트에 참가하는 것에 대해 긴장해요."
    }
  ]
}


# CSV 파일로 변환

In [15]:
for sample in sample_list:
    sample_json = json.loads(sample)
    content = sample_json["response"]["body"]["choices"][0]["message"]["content"]
    content_json = json.loads(content)
    # print(json.dumps(content_json, indent=2, ensure_ascii=False))
    # break

In [16]:
content = sample_json["response"]["body"]["choices"][0]["message"]["content"]
content_json = json.loads(content)

In [17]:
def JsonToCSV(data:json, file_name:str) -> json:
    data_list = []

    for sample in data:
        # json 변환환
        sample_json = json.loads(sample)
        content = sample_json["response"]["body"]["choices"][0]["message"]["content"]
        content_json = json.loads(content)

        # 학습 목표
        goal = "\n".join(content_json["goal"])

        # 퀴즈 1
        quiz1 = content_json["preview_quizzes"][0]
        quiz1_data = {
            "Quiz 1": quiz1["fill_in_blank_quiz"],
            "Korean 1": quiz1["korean_translation"],
            "Option 1-1": quiz1["options"][0],
            "Option 1-2": quiz1["options"][1],
            "Option 1-3": quiz1["options"][2],
            "Answer 1": quiz1["answer"],
            "Index 1": quiz1["answer_index"]
        }

        # 퀴즈 2
        quiz2 = content_json["preview_quizzes"][1]
        quiz2_data = {
            "Quiz 2": quiz2["fill_in_blank_quiz"],
            "Korean 2": quiz2["korean_translation"],
            "Option 2-1": quiz2["options"][0],
            "Option 2-2": quiz2["options"][1],
            "Option 2-3": quiz2["options"][2],
            "Answer 2": quiz2["answer"],
            "Index 2": quiz2["answer_index"]
        }

        # script
        sample_script_eng = content_json["sample_script"][0]["sample_script_eng"]
        sample_script_kor = content_json["sample_script"][0]["sample_script_kor"]

        # 데이터 만들기
        csv_data = {
            "학습목표": goal,
            **quiz1_data,
            **quiz2_data,
            "샘플 스크립트 (영어)": sample_script_eng,
            "샘플 스크립트 (한국어)": sample_script_kor
        }

        data_list.append(csv_data)

    df = pd.DataFrame(data_list)

    # CSV 저장
    df.to_csv(f"{file_name}.csv", index=False, encoding='utf-8-sig')

    print(f"CSV 변환 완료 파일명: {file_name}.csv")


In [18]:
name = 'PreviewQuiz_Structure_High'
JsonToCSV(sample_list, name)

CSV 변환 완료 파일명: PreviewQuiz_Structure_High.csv


In [21]:
len(df)

329

In [20]:
df = pd.read_csv(f'{name}.csv')
df.head(1)

,학습목표,Quiz 1,Korean 1,Option 1-1,Option 1-2,Option 1-3,Answer 1,Index 1,Quiz 2,Korean 2,Option 2-1,Option 2-2,Option 2-3,Answer 2,Index 2,샘플 스크립트 (영어),샘플 스크립트 (한국어)
0,전치사의 목적어로서의 동명사를 사용할 수 있어요.,I enjoy _____ in team sports.,나는 팀 스포츠에서 노는 것을 즐겨요.,play,playing,played,playing,2,She is excited about _____ goals in matches.,그녀는 경기에서 골 넣는 것에 신나해요.,scoring,score,scored,scoring,1,A: What are you excited about these days? B: I...,A: 요즘 무엇에 신나해요? B: 축제에서 춤추는 것에 신나해요. A: 왜 그 사람...


# Cancelling a Batch

In [19]:
# cancel_batch = client.batches.cancel("")
# batch_json = json.loads(cancel_batch.json())
# batch_ouput = json.dumps(batch_json, ensure_ascii=False, indent=2)
# print(batch_ouput)